In [1]:
# Force Keras to use the CPU
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"] = ""

# NOTE: This code block silences memory allocation warnings, which are empirically annoying.
# It's a good idea to have it on regularly when you're making changes.
import tensorflow as tf
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

# Our imports
from keras.models import Sequential
from keras.layers import Dense
import sqlite3
import pickle
from datetime import datetime
from keras import regularizers

# old imports
from sklearn.svm import LinearSVC
from sklearn import preprocessing
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
import collections

/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
encoded = np.load("./npy_data/data_encoded_d.npy")

In [3]:
blood_types = np.load('./npy_data/blood_types.npy')

In [8]:
np.bincount(blood_types)

array([49, 30])

In [9]:
X_train, X_test, y_train, y_test = train_test_split(encoded, blood_types, test_size=0.2)

In [10]:
X_train.shape

(63, 25019446)

In [27]:
np.bincount(y_train)

array([27, 21])

In [28]:
model = Sequential()
model.add(Dense(10, input_shape=(encoded.shape[1],), kernel_initializer='uniform', activation='relu'))
model.add(Dense(5, kernel_initializer='uniform', activation='relu'))
model.add(Dense(5, kernel_initializer='uniform', activation='relu'))
model.add(Dense(1, kernel_initializer='uniform', activation='sigmoid'))

In [30]:
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 10)                250194470 
_________________________________________________________________
dense_6 (Dense)              (None, 5)                 55        
_________________________________________________________________
dense_7 (Dense)              (None, 5)                 30        
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 6         
Total params: 250,194,561
Trainable params: 250,194,561
Non-trainable params: 0
_________________________________________________________________
None


In [33]:
from keras import optimizers
model.compile(loss='binary_crossentropy', optimizer=optimizers.adam(lr=0.001), metrics=['accuracy'])

In [34]:
zeros = np.argwhere(blood_types == 0)[:30]
ones = np.argwhere(blood_types == 1)[:30]
keep_idx = np.concatenate((zeros, ones))
np.random.shuffle(keep_idx)

In [35]:
blood_keep = blood_types[keep_idx]
encoded_keep = encoded[keep_idx]

In [36]:
encoded_keep = encoded_keep.ravel().reshape(60, 25019446)

In [37]:
blood_keep = np.reshape(blood_keep, len(blood_keep))

In [38]:
X_train, X_test, y_train, y_test = train_test_split(encoded_keep, blood_keep, test_size=0.2)

In [ ]:
y_test

array([0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1])

In [ ]:
history = model.fit(X_train, 
                     y_train, 
                     validation_data=(X_test, y_test),
                     epochs = 50, 
                     batch_size = 10,
                     verbose = 1)

Train on 48 samples, validate on 12 samples
Epoch 1/50
48/48 [==============================] - 57s 1s/step - loss: 7.3875 - acc: 0.5417 - val_loss: 10.7454 - val_acc: 0.3333
Epoch 2/50
48/48 [==============================] - 41s 857ms/step - loss: 7.3875 - acc: 0.5417 - val_loss: 10.7454 - val_acc: 0.3333
Epoch 3/50
48/48 [==============================] - 41s 864ms/step - loss: 7.3875 - acc: 0.5417 - val_loss: 10.7454 - val_acc: 0.3333
Epoch 4/50
48/48 [==============================] - 40s 830ms/step - loss: 7.3875 - acc: 0.5417 - val_loss: 10.7454 - val_acc: 0.3333
Epoch 5/50
48/48 [==============================] - 40s 840ms/step - loss: 7.3875 - acc: 0.5417 - val_loss: 10.7454 - val_acc: 0.3333
Epoch 6/50
48/48 [==============================] - 41s 849ms/step - loss: 7.3875 - acc: 0.5417 - val_loss: 10.7454 - val_acc: 0.3333
Epoch 7/50
48/48 [==============================] - 40s 840ms/step - loss: 7.3875 - acc: 0.5417 - val_loss: 10.7454 - val_acc: 0.3333
Epoch 8/50
48/48 [===

In [ ]:
model.predict(X_test)